## 數據整合平台 v15 - 穩定版執行器
這個筆記本是執行數據整合平台的官方、穩定入口。
**核心優勢**：使用 `Poetry` 來安裝 `pyproject.toml` 中鎖定版本的依賴，並透過 `git clone` 自動獲取最新程式碼，徹底解決因環境不一致或手動設定錯誤導致的執行失敗問題。

In [ ]:
#@markdown ## 執行參數設定
#@markdown 請在下方輸入您的執行參數，若不修改則使用預設值。
project_folder = "MyTaifexDataProject" #@param {type:"string"}
database_name = "processed_data.duckdb" #@param {type:"string"}
log_name = "pipeline.log" #@param {type:"string"}
target_zip_files = "" #@param {type:"string"}
#@markdown *若要指定多個ZIP檔，請用逗號分隔，例如 `file1.zip,file2.zip`。留空則代表處理全部。*

print("--- 參數確認 ---")
print(f"專案資料夾: {project_folder}")
print(f"輸出資料庫: {database_name}")
print(f"輸出日誌檔: {log_name}")
print(f"指定ZIP檔: {'全部' if not target_zip_files else target_zip_files}")
print("--------------------\n")

# ======================================================================
#                      穩健執行區 (單一儲存格)
# ======================================================================
import os
import sys
from google.colab import drive

# 步驟 1: 掛載 Google Drive
try:
    print("⏳ 正在請求掛載 Google Drive...")
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive 掛載成功！")
except Exception as e:
    print(f"❌ Google Drive 掛載失敗: {e}")
    sys.exit("請檢查您的 Drive 授權設定。")

# 步驟 2: 下載/更新專案程式碼
print("\n⏳ 正在準備專案程式碼...")
REPO_PATH = '/content/taifexd-date'
REPO_URL = 'https://github.com/hsp1234-web/taifexd-date.git'

if os.path.exists(REPO_PATH):
    print(f"  - 找到現有目錄，執行 git pull 更新...")
    os.chdir(REPO_PATH)
    !git pull
else:
    print(f"  - 執行 git clone 下載專案...")
    !git clone -q {REPO_URL} {REPO_PATH}

os.chdir(REPO_PATH)
print(f"✅ 專案程式碼準備完成！目前工作目錄: {os.getcwd()}\n")

# 步驟 3: 同步鎖定檔並安裝依賴 (*** 已修正此區塊 ***)
print("\n⏳ 正在同步 lock 檔案並安裝依賴...")
!pip install poetry -q
print("  - 執行 poetry lock 來解決不一致問題...")
!poetry lock --no-update
print("  - 執行 poetry install 來安裝套件...")
!poetry install --no-root --without dev
print("✅ 所有精確版本的 Python 套件均已安裝完成！\n")

# 步驟 4: 建構並執行指令
print("\n🚀 即將啟動數據整合平台...")
command = (
    f"poetry run python main.py "
    f"--project-folder-name='{project_folder}' "
    f"--database-name='{database_name}' "
    f"--log-name='{log_name}' "
    f"--zip-files='{target_zip_files}'"
)

print(f"  - 將執行的最終指令:")
print(f"    {command}")
print("-" * 20)

!{command}

print("-" * 20)
print("🎉 執行完畢！")
print(f"ℹ️  請檢查您的 Google Drive/{project_folder} 資料夾以確認產出檔案。")